## NOISY statistics

This notebook contains functions used for the analysis and visualization of the NOISY program benchmark. 

In [ ]:
PREFIX = "/data/ruslan_gumerov/"

In [ ]:
import os
import re
import glob
import numpy as np
import polars as pl
import pandas as pd
import seaborn as sns
from scipy import stats
from Bio import AlignIO
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
from plotly.subplots import make_subplots
from sklearn.neighbors import KernelDensity
from NOISY_output_analysis_functions import *

In [ ]:
#CALCULATE ALL RF STATISTICS

if __name__ == "__main__":
    
    for group in ["fungi", "metazoa", "viridiplantae", "proteobacteria", "actinobacteria", "archaea"]:
        print(group)
        PATH_TO_ORIGINAL_DIR = f"{PREFIX}{group}/{group}_results_original/"
        PATH_TO_DENOISED_DIR = f"{PREFIX}{group}/{group}_results_denoised/"
        PATH_TO_INITIAL_ALIGNMNENTS = f"{PREFIX}{group}/{group}_fasta_aligned/"

        #filter out bad files 
        if group == "archaea":
            filter_rf_output(PATH_TO_DENOISED_DIR, remove = True)
            filter_rf_output(PATH_TO_ORIGINAL_DIR, remove = True)
        else:
            filter_rf_output(PATH_TO_DENOISED_DIR, remove = False)
            filter_rf_output(PATH_TO_ORIGINAL_DIR, remove = False)

        #get RF vaLues
        rf_df_denoised = get_rf_from_log([x for x in  glob.glob(PATH_TO_DENOISED_DIR + "*tmp")])
        rf_df_original = get_rf_from_log([x for x in  glob.glob(PATH_TO_ORIGINAL_DIR + "*tmp")])

        #get alignment width and length
        fasta_df_denoised = get_alignment_width_length([x for x in glob.glob(PATH_TO_DENOISED_DIR + "*fas")])
        fasta_df_original = get_alignment_width_length([x for x in glob.glob(PATH_TO_INITIAL_ALIGNMNENTS + "*fasta")])

        #concat RF values and width/length of files
        rf_df_denoised = pd.concat([fasta_df_denoised, rf_df_denoised], axis = 1).dropna()
        rf_df_original = pd.concat([fasta_df_original, rf_df_original], axis = 1).dropna()

        #harmonize filenames
        rf_df_original = rf_df_original[rf_df_original.index.isin(rf_df_denoised.index)]
        rf_df_denoised = rf_df_denoised[rf_df_denoised.index.isin(rf_df_original.index)]

        
        #get domain info
        domains_table = pd.read_csv(f"{PREFIX}{group}/{group}_proteins_species_orders_mapping.csv", index_col=0)
        domains_table["apply"] = domains_table["apply"].map(clean_str)
        domains_table["taxa"] = domains_table[domains_table.columns[-1]].map(clean_str)
        PATH_TO_PROTEIN_DOMAIN_FILE = f"{PREFIX}{group}/{group}_domains_and_proteins_csv/"
        domains_table["domains_amount"] = domains_table.apply(lambda row: get_domains_number(PATH_TO_PROTEIN_DOMAIN_FILE, row['apply'], row['taxa']),axis=1)

        rf_out_df = create_difference_dataframe(rf_df_denoised, rf_df_original)
        rf_out_df.index = rf_out_df.index.str.split("_").str[0].astype(int)
        rf_out_df.sort_index(inplace = True)
        rf_out_df["domains_amount"] = domains_table["domains_amount"]

        
        #filter our strange domains
        rf_out_df = rf_out_df[rf_out_df["domains_amount"].str.len() == 1]
        
        rf_out_df["domains_amount"] = [list(map(str,x))[0] for x in rf_out_df["domains_amount"]]
        rf_out_df.to_csv(f"{PREFIX}{group}/{group}_RF_statistics_df.csv")


In [ ]:
#READ PRECALCULATED DF STATISTICS
list_of_df = []
for group in ["fungi", "metazoa", "viridiplantae", "proteobacteria", "actinobacteria", "archaea"]:
    print(group)
    group_out = pd.read_csv(f"{PREFIX}{group}/{group}_RF_statistics_df.csv")
    group_out["cutoff_percent"] = (group_out["RF_original_length"] - group_out["RF_denoised_length"]) / group_out["RF_original_length"] 
    list_of_df.append(group_out)

eukaryota = pd.concat([list_of_df[0], list_of_df[1], list_of_df[2]])
prokaryota = pd.concat([list_of_df[3], list_of_df[4], list_of_df[5]])
total = pd.concat([list_of_df[0], list_of_df[1], list_of_df[2], list_of_df[3], list_of_df[4], list_of_df[5]])


In [ ]:
## LENGTH
def noisy_density_before_after(list_of_df, name = "Not specified"):
    fungi = pd.DataFrame({ "log_length_original" : np.log10(list_of_df[0]["RF_original_length"]),
                         "log_length_denoised" : np.log10(list_of_df[0]["RF_denoised_length"])})
    sns_plot = sns.kdeplot(data=fungi, x="log_length_original")
    sns_plot = sns.kdeplot(data=fungi, x="log_length_denoised")
        
    metazoa = pd.DataFrame({ "log_length_original" : np.log10(list_of_df[1]["RF_original_length"]),
                         "log_length_denoised" : np.log10(list_of_df[1]["RF_denoised_length"])})
    sns_plot = sns.kdeplot(data=metazoa, x="log_length_original")
    sns_plot = sns.kdeplot(data=metazoa, x="log_length_denoised")
    
    viridiplantae = pd.DataFrame({ "log_length_original" : np.log10(list_of_df[2]["RF_original_length"]),
                         "log_length_denoised" : np.log10(list_of_df[2]["RF_denoised_length"])})
    sns_plot = sns.kdeplot(data=viridiplantae, x="log_length_original")
    sns_plot = sns.kdeplot(data=viridiplantae, x="log_length_denoised")
    
    proteobacteria = pd.DataFrame({ "log_length_original" : np.log10(list_of_df[3]["RF_original_length"]),
                         "log_length_denoised" : np.log10(list_of_df[3]["RF_denoised_length"])})
    sns_plot = sns.kdeplot(data=proteobacteria, x="log_length_original")
    sns_plot = sns.kdeplot(data=proteobacteria, x="log_length_denoised")
    
    actinobacteria = pd.DataFrame({ "log_length_original" : np.log10(list_of_df[4]["RF_original_length"]),
                         "log_length_denoised" : np.log10(list_of_df[4]["RF_denoised_length"])})
    sns_plot = sns.kdeplot(data=actinobacteria, x="log_length_original")
    sns_plot = sns.kdeplot(data=actinobacteria, x="log_length_denoised")
    
    archaea = pd.DataFrame({ "log_length_original" : np.log10(list_of_df[5]["RF_original_length"]),
                         "log_length_denoised" : np.log10(list_of_df[5]["RF_denoised_length"])})
    sns_plot = sns.kdeplot(data=archaea, x="log_length_original")
    sns_plot = sns.kdeplot(data=archaea, x="log_length_denoised")
    
    fig = make_subplots()
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[0].get_data()[0], 
            y=sns_plot.get_lines()[0].get_data()[1], 
            mode="lines", 
            marker_color='#FFBC42',
            name="fungi original",
            line = dict(width = 1.5)))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[1].get_data()[0], 
            y=sns_plot.get_lines()[1].get_data()[1], 
            mode="lines", 
            marker_color='#FFBC42',
            name="fungi denoised",
            line = dict(width = 1.5, dash='dash')))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[2].get_data()[0], 
            y=sns_plot.get_lines()[2].get_data()[1], 
            mode="lines", 
            marker_color='#D81159',
            name="metazoa original",
            line = dict(width = 1.5)))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[3].get_data()[0], 
            y=sns_plot.get_lines()[3].get_data()[1], 
            mode="lines", 
            marker_color='#D81159',
            name="metazoa denoised",
            line = dict(width = 1.5, dash='dash')))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[4].get_data()[0], 
            y=sns_plot.get_lines()[4].get_data()[1], 
            mode="lines", 
            marker_color='#8F2D56',
            name="viridiplantae original",
            line = dict(width = 1.5)))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[5].get_data()[0], 
            y=sns_plot.get_lines()[5].get_data()[1], 
            mode="lines", 
            marker_color='#8F2D56',
            name="viridiplantae denoised",
            line = dict(width = 1.5, dash='dash')))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[6].get_data()[0], 
            y=sns_plot.get_lines()[6].get_data()[1], 
            mode="lines", 
            marker_color='#58586B',
            name="proteobacteria original",
            line = dict(width = 1.5)))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[7].get_data()[0], 
            y=sns_plot.get_lines()[7].get_data()[1], 
            mode="lines", 
            marker_color='#58586B',
            name="proteobacteria denoised",
            line = dict(width = 1.5, dash='dash')))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[8].get_data()[0], 
            y=sns_plot.get_lines()[8].get_data()[1], 
            mode="lines", 
            marker_color='#218380',
            name="actinobacteria original",
            line = dict(width = 1.5)))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[9].get_data()[0], 
            y=sns_plot.get_lines()[9].get_data()[1], 
            mode="lines", 
            marker_color='#218380',
            name="actinobacteria denoised",
            line = dict(width = 1.5, dash='dash')))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[10].get_data()[0], 
            y=sns_plot.get_lines()[10].get_data()[1], 
            mode="lines", 
            marker_color='#73D2DE',
            name="archaea original",
            line = dict(width = 1.5)))
    
    fig.add_trace(
        go.Scatter(
            x=sns_plot.get_lines()[11].get_data()[0], 
            y=sns_plot.get_lines()[11].get_data()[1], 
            mode="lines", 
            marker_color='#73D2DE',
            name="archaea denoised",
            line = dict(width = 1.5, dash='dash')))
    
    fig.update_layout(
        title= f"Плотность длин для разных классов",
        xaxis_title="Длина последовательности (log10)",
        template="none"
    )
    
    fig.update_yaxes(title_text="Количество последовательностей", rangemode='tozero', secondary_y=False)
    fig.update_yaxes(title_text="Плотность", secondary_y=True, showgrid=False,
                     visible=False, showticklabels=False, rangemode='tozero', tickmode="auto")
    
    return fig

fig1 = noisy_density_before_after(list_of_df, group)
fig1
fig1.write_image(f"{PREFIX}images/total_length_density.pdf")

In [ ]:
#NOISY CUTOFF DENSITY

#["fungi", "metazoa", "viridiplantae", "proteobacteria", "actinobacteria", "archaea"]:

def noisy_rf_difference_density(list_of_df, name = "Not specified"):
    
    fig = make_subplots()
    
    fig.add_trace(
        go.Histogram(
            x=list_of_df[5]["RF_original"] - list_of_df[5]["RF_denoised"],
            name='original',
            marker_color='#636EFA',
            nbinsx=50,
            opacity=0.75),
        secondary_y=False)

    fig.update_traces(marker_line_width=1,marker_line_color="black")

    fig.update_layout(
        title= f"Изменение в RF для {name}",
        yaxis_title="Число последовательностей",
        xaxis_title="разность RF",
        template="none",
    )
    
    fig.update_yaxes(title_text="Число последовательностей", rangemode='tozero', secondary_y=False)
    fig.update_yaxes(title_text="Плотность", secondary_y=True, showgrid=False,
                     visible=False, showticklabels=False, rangemode='tozero', tickmode="auto")
        
    return fig

fig3 = noisy_rf_difference_density(list_of_df, "archaea")
fig3
fig3.write_image(f"{PREFIX}images/{group}_rf_changes_histogram.pdf")

In [ ]:
#LENGTH
#Warning! Palette and order is changing during conversion from sns.kdeplot.get_lines() and go.Scatter
#Need to set up colors manually
def RF_length_KDE(rf_out_df, name = "Not specified"):
    rf_out_df["RF_original_length_log"] = np.log10(rf_out_df["RF_original_length"])
    sns_plot = sns.kdeplot(data=rf_out_df, x="RF_original_length_log", hue="RF_bool")
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[2].get_data()[0], 
                             y=sns_plot.get_lines()[2].get_data()[1], 
                             mode="lines", name="Исходный RF выше",
                             line=dict(color="#ff595e")))                             
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[0].get_data()[0], 
                             y=sns_plot.get_lines()[0].get_data()[1], 
                             mode="lines", name="Исходный RF ниже",
                             line=dict(color="#8ac926")))
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[1].get_data()[0], 
                             y=sns_plot.get_lines()[1].get_data()[1], 
                             mode="lines", name="Без изменений",
                             line=dict(color="#ffca3a")))

    fig.update_layout(
        title= f"Распределение RF для разных классов в зависимости от длины последовательностей в {name}",
        xaxis_title="Длина последовательности (log10)",
        yaxis_title="Плотность",
        template="none",
    )
    
    del sns_plot
    
    return fig

group = "fungi"
fig4 = RF_length_KDE(list_of_df[0], group)
#ffca3a 8ac926 ff595e
#["fungi", "metazoa", "viridiplantae", "proteobacteria", "actinobacteria", "archaea"]
#fig4.write_image(f"{PREFIX}images/{group}_rf_classes_length_density.pdf")
fig4

In [ ]:
#WIDTH
#Warning! Palette and order is changing during conversion from sns.kdeplot.get_lines() and go.Scatter
#Need to set up colors manually
def RF_width_KDE(rf_out_df, name = "Not specified"):
    #rf_out_df["RF_original_length_log"] = np.log(rf_out_df["RF_original_length"])
    sns_plot = sns.kdeplot(data=rf_out_df, x="RF_original_width", hue="RF_bool", clip=(0.0, 350))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[0].get_data()[0], 
                             y=sns_plot.get_lines()[0].get_data()[1], 
                             mode="lines", name="Исходный RF ниже",
                             line=dict(color="#8ac926")))
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[1].get_data()[0], 
                             y=sns_plot.get_lines()[1].get_data()[1], 
                             mode="lines", name="Без изменений",
                             line=dict(color="#ffca3a")))
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[2].get_data()[0], 
                             y=sns_plot.get_lines()[2].get_data()[1], 
                             mode="lines", name="Исходный RF выше",
                             line=dict(color="#ff595e")))

    fig.update_layout(
        title= f"Распределение RF для разных классов в зависимости от количества последовательностей в {name}",
        xaxis_title="Количество последовательностей",
        yaxis_title="Плотность",
        template="none",
    )
    
    del sns_plot
        
    return fig

group = "archaea"
fig5 = RF_width_KDE(list_of_df[5], group)
#ffca3a 8ac926 ff595e
#["fungi", "metazoa", "viridiplantae", "proteobacteria", "actinobacteria", "archaea"]
#fig5.write_image(f"{PREFIX}images/{group}_rf_classes_width_density.pdf")
fig5

In [ ]:
## RF CUTOFF PERCENT
#Warning! Palette and order is changing during conversion from sns.kdeplot.get_lines() and go.Scatter
#Need to set up colors manually
def RF_cutoff_KDE(rf_out_df, name):
    rf_out_df["cutoff_percent_log"] = rf_out_df["cutoff_percent"]
    sns_plot = sns.kdeplot(data=rf_out_df, x="cutoff_percent_log", hue="RF_bool")
    
    print(sum(rf_out_df["cutoff_percent_log"])/len(rf_out_df["cutoff_percent_log"]))
    print(rf_out_df["cutoff_percent_log"].median())
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[0].get_data()[0], 
                             y=sns_plot.get_lines()[0].get_data()[1], 
                             mode="lines", name="Исходный RF ниже",
                             line=dict(color="#8ac926")))
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[1].get_data()[0], 
                             y=sns_plot.get_lines()[1].get_data()[1], 
                             mode="lines", name="Без изменений",
                             line=dict(color="#ffca3a")))
    fig.add_trace(go.Scatter(x=sns_plot.get_lines()[2].get_data()[0], 
                             y=sns_plot.get_lines()[2].get_data()[1], 
                             mode="lines", name="Исходный RF выше",
                             line=dict(color="#ff595e")))

    fig.update_layout(
        title= f"Распределение RF для разных классов в зависимости от доли отрезанной длины в {name}",
        xaxis_title="Процент отрезанной длины (log10)",
        yaxis_title="Плотность",
        template="none",
    )
    
    del sns_plot
    
    return fig
 
group = "archaea"
fig6 = RF_cutoff_KDE(list_of_df[1], group)
#ffca3a 8ac926 ff595e
#["fungi", "metazoa", "viridiplantae", "proteobacteria", "actinobacteria", "archaea"]
#fig6.write_image(f"{PREFIX}images/{group}_rf_classes_cutoff_density.pdf")
fig6

In [ ]:
#READ BOOTSTRAP LOGS
def bootstrap_df_create(PATH_TO_BOOTSTRAP_RESULTS, dataframe):
    file_list = []
    branch_list = []
    bootstrap_list = []
    for file in os.listdir(PATH_TO_BOOTSTRAP_RESULTS):
        if file.endswith("tree.nwk"):
            with open(f"{PATH_TO_BOOTSTRAP_RESULTS}{file}", "r") as inp:
                first_line = inp.readline()
                matches = re.findall(r'\)(\d+):', first_line)
                file_list.append(file)
                branch_list.append(len(matches))
                bootstrap_list.append(sum(list(map(int, matches))))
                
    dataframe["file"] = file_list
    dataframe["branch_length"] = branch_list
    dataframe["bootstrap"] = bootstrap_list
    return dataframe

bootstrap_df_list = []

#CREATE LIST OF DATAFRAMES WITH RF
for group in ["fungi", "metazoa", "viridiplantae", "proteobacteria", "actinobacteria", "archaea"]:
    print(group)
    PATH_TO_ORIGINAL_DIR = f"{PREFIX}{group}/{group}_results_original_bootstrap/"
    PATH_TO_DENOISED_DIR = f"{PREFIX}{group}/{group}_results_denoised_bootstrap/"
    PATH_TO_INITIAL_ALIGNMNENTS = f"{PREFIX}{group}/{group}_fasta_aligned/"

    #filter out bad files 
    if group == "archaea":
        filter_rf_output(PATH_TO_DENOISED_DIR, remove = True)
        filter_rf_output(PATH_TO_ORIGINAL_DIR, remove = True)
    else:
        filter_rf_output(PATH_TO_DENOISED_DIR, remove = False)
        filter_rf_output(PATH_TO_ORIGINAL_DIR, remove = False)

    #get RF vaLues
    rf_df_denoised = get_rf_from_log([x for x in  glob.glob(PATH_TO_DENOISED_DIR + "*tmp")])
    rf_df_original = get_rf_from_log([x for x in  glob.glob(PATH_TO_ORIGINAL_DIR + "*tmp")])

    #get alignment width and length
    fasta_df_denoised = get_alignment_width_length([x for x in glob.glob(PATH_TO_DENOISED_DIR + "*fas")])
    fasta_df_original = get_alignment_width_length([x for x in glob.glob(PATH_TO_INITIAL_ALIGNMNENTS + "*fasta")])

    #concat RF values and width/length of files
    rf_df_denoised = pd.concat([fasta_df_denoised, rf_df_denoised], axis = 1).dropna()
    rf_df_original = pd.concat([fasta_df_original, rf_df_original], axis = 1).dropna()

    #harmonize filenames
    rf_df_original = rf_df_original[rf_df_original.index.isin(rf_df_denoised.index)]
    rf_df_denoised = rf_df_denoised[rf_df_denoised.index.isin(rf_df_original.index)]


    #get domain info
    rf_out_df = create_difference_dataframe(rf_df_denoised, rf_df_original)
    rf_out_df.index = rf_out_df.index.str.split("_").str[0].astype(int)
    rf_out_df.sort_index(inplace = True)

    #filter our strange domains
    original_df = pd.DataFrame({"file" : [], "branch_length" : [], "bootstrap" : []})
    denoised_df = pd.DataFrame({"file" : [], "branch_length" : [], "bootstrap" : []})

    original_df = bootstrap_df_create(PATH_TO_ORIGINAL_DIR, original_df)
    denoised_df = bootstrap_df_create(PATH_TO_DENOISED_DIR, denoised_df)

    original_df.sort_values("file", inplace=True)
    denoised_df.sort_values("file", inplace=True)

    original_df.index = original_df["file"].str.split("_").str[0].astype(int)
    denoised_df.index = denoised_df["file"].str.split("_").str[0].astype(int)
    denoised_df.drop(["file"], axis = 1, inplace=True)
    original_df.drop(["file"], axis = 1, inplace=True)

    original_df.sort_index(inplace = True)
    denoised_df.sort_index(inplace = True)

    rf_out_df["bootstrap_original"] = original_df["bootstrap"]
    rf_out_df["bootstrap_denoised"] = denoised_df["bootstrap"]
    rf_out_df["branch_length_original"] = original_df["branch_length"]
    rf_out_df["branch_length_denoised"] = denoised_df["branch_length"]
    
    bootstrap_df_list.append(rf_out_df)

    print(rf_out_df)



In [ ]:
#CALCULATE STATISTICS FOR BOOTSTRAP

eukaryota.query("RF_original_width < 38 & bootstrap_denoised <= bootstrap_original & RF_code == 'NOISY makes worse'")

